<a href="https://colab.research.google.com/github/yellowflickerbeat/FakeNews_Detection/blob/main/FakeNews_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
#stopwords removes all the repetative words which don't substance to the text
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
#WordNetLemmatizer converts words into their roots considering its context
from sklearn.feature_extraction.text import TfidfVectorizer
#TfidfVectorizer measures importance of diff words
from sklearn.model_selection import train_test_split
#splits data into test and train
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [7]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [8]:
print(stopwords.words('english'))
#examples of all stopwords

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Pre-Processing

In [13]:
#mounting the mandatory datasets
news = pd.read_table('/content/train.tsv')

In [14]:
news.shape
#getting to know the contents' size

(30000, 6)

In [17]:
#printing the first 10 rows and columns
news = news.rename(columns={'Unnamed:0' : 'id'})
news.head(10)

,Unnamed: 0,title,text,subject,date,label
0,2619,Ex-CIA head says Trump remarks on Russia inter...,Former CIA director John Brennan on Friday cri...,politicsNews,"July 22, 2017",1
1,16043,YOU WON’T BELIEVE HIS PUNISHMENT! HISPANIC STO...,How did this man come to OWN this store? There...,Government News,"Jun 19, 2017",0
2,876,Federal Reserve governor Powell's policy views...,President Donald Trump on Thursday tapped Fede...,politicsNews,"November 2, 2017",1
3,19963,SCOUNDREL HILLARY SUPPORTER STARTS “TrumpLeaks...,Hillary Clinton ally David Brock is offering t...,left-news,"Sep 17, 2016",0
4,10783,NANCY PELOSI ARROGANTLY DISMISSES Questions on...,Pleading ignorance is a perfect ploy for Nancy...,politics,"May 26, 2017",0
5,18522,EU's Tusk appealed to Rajoy to avoid escalatio...,European Council President Donald Tusk appeale...,worldnews,"October 2, 2017",1
6,270,Country Guitarist Who Survived Vegas Shooting ...,"Caleb Keeter, a lifelong proponent of the Seco...",News,"October 2, 2017",0
7,7628,Clinton says 'there is no case here' in FBI em...,Democratic presidential candidate Hillary Clin...,politicsNews,"October 31, 2016",1
8,9599,ABC NEWS REPORTS: Las Vegas Massacre Suspect’s...,The investigation into the Las Vegas massacre ...,politics,"Oct 25, 2017",0
9,11234,BONKERS BERNIE SANDERS: Prioritizing Jobs Over...,https://www.youtube.com/watch?v=GPqQIlWksbgVer...,politics,"Apr 1, 2017",0


In [16]:
#find out missing values in dataset
news.isnull().sum()

,0
Unnamed: 0,0
title,0
text,0
subject,0
date,0
label,0


In [30]:
#renaming a column in dataset to uniquely identify eash item
news = news.rename(columns={'Unnamed: 0' : 'id'})

In [28]:
#removing the previous values in 'label' column
news = news.drop('label', axis=1)

**Lemmatization**

Lemmatization helps to reduce dimensionality esp as we are using logistic regression

Unlike stemming, it reduces the words too its base form only after considering the words' contexts

In [33]:
lemma = WordNetLemmatizer()

In [35]:
def lemmatization(content):
    lemmatizer = WordNetLemmatizer()
    # Remove all non-alphabetic characters & convert to lowercase
    lemmatized_content = re.sub('[^a-zA-Z]', ' ', content)
    lemmatized_content = lemmatized_content.lower()
    # Split the content into words
    lemmatized_content = lemmatized_content.split()
    # Remove stopwords and apply lemmatization
    lemmatized_content = [lemmatizer.lemmatize(word) for word in lemmatized_content if word not in stopwords.words('english')]
    # Join the words back into a single string
    lemmatized_content = ' '.join(lemmatized_content)
    return lemmatized_content


In [37]:
nltk.download('wordnet')
news['title'] = news['title'].apply(lemmatization)
news['text'] = news['text'].apply(lemmatization)

[nltk_data] Downloading package wordnet to /root/nltk_data...
